In [1]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, classification_report
from imblearn.over_sampling import SMOTE
import mlflow
import mlflow.sklearn
import optuna
import dagshub
import pickle


dagshub.init(repo_owner='satyajeetrai007', repo_name='Youtube-Comment-Sentiment-Analysis', mlflow=True)
mlflow.set_experiment("Algorithms with HyperParameter tuning")

df = pd.read_csv('data_preprocessed.csv').dropna().drop_duplicates()
df.shape

c:\Users\satya\OneDrive\Desktop\youtube-comment-senitment-analysis\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Accessing as satyajeetrai007

Initialized MLflow to track repo "satyajeetrai007/Youtube-Comment-Sentiment-Analysis"

Repository satyajeetrai007/Youtube-Comment-Sentiment-Analysis initialized!

(36243, 2)

In [2]:
df = df.dropna(subset=['category'])
X = df['clean_comment']
y = df['category']

# Step 2: Perform the train-test split FIRST to prevent data leakage
# We use stratify to maintain the original class distribution in both sets.
X_train_text, X_test_text, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Step 3: TF-IDF vectorizer setup
# Fit the vectorizer ONLY on the training data
vectorizer = TfidfVectorizer(ngram_range=(1, 3), max_features=2000)
X_train_vec = vectorizer.fit_transform(X_train_text)

# Transform the test data using the already-fitted vectorizer
X_test_vec = vectorizer.transform(X_test_text)

# Step 4: Apply SMOTE to the TRAINING data ONLY
smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train_vec, y_train)

# --- The test sets (X_test_vec, y_test) remain untouched by SMOTE ---

# Function to log results in MLflow (no changes needed here)
def log_mlflow(model_name, model, X_train, X_test, y_train, y_test):
    with mlflow.start_run():
        # Log model type
        mlflow.set_tag("mlflow.runName", f"{model_name}_SMOTE_TFIDF_Trigrams_Corrected")
        mlflow.set_tag("experiment_type", "algorithm_comparison")

        # Log algorithm name as a parameter
        mlflow.log_param("algo_name", model_name)

        # Train model
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)

        # Log accuracy
        accuracy = accuracy_score(y_test, y_pred)
        mlflow.log_metric("accuracy", accuracy)

        # Log classification report
        classification_rep = classification_report(y_test, y_pred, output_dict=True)
        for label, metrics in classification_rep.items():
            if isinstance(metrics, dict):
                for metric, value in metrics.items():
                    mlflow.log_metric(f"{label}_{metric}", value)

        # Log the model
        model_filename = f"{model_name}_model.pkl"
        with open(model_filename, "wb") as f:
            pickle.dump(model, f)
        mlflow.log_artifact(model_filename, "model")

# Step 5: Optuna objective function for KNN
# It will use the correctly processed data from the global scope
def objective_knn(trial):
    n_neighbors = trial.suggest_int('n_neighbors', 3, 30)
    p = trial.suggest_categorical('p', [1, 2])

    model = KNeighborsClassifier(n_neighbors=n_neighbors, p=p)
    model.fit(X_train_resampled, y_train_resampled)
    
    # Evaluate on the original, unmodified test set
    y_pred = model.predict(X_test_vec)
    return accuracy_score(y_test, y_pred)

# Step 6: Run Optuna for KNN, log the best model only
def run_optuna_experiment():
    study = optuna.create_study(direction="maximize")
    study.optimize(objective_knn, n_trials=30)

    best_params = study.best_params
    best_model = KNeighborsClassifier(n_neighbors=best_params['n_neighbors'], p=best_params['p'])

    # Log the best model with MLflow, using the resampled training data
    # and the original test data.
    log_mlflow("KNN", best_model, X_train_resampled, X_test_vec, y_train_resampled, y_test)

# Run the experiment
run_optuna_experiment()

[I 2025-08-27 20:18:47,700] A new study created in memory with name: no-name-49fe429d-32e2-4fe7-96f6-7f2c0b2abb0f
[I 2025-08-27 20:19:20,141] Trial 0 finished with value: 0.3596358118361153 and parameters: {'n_neighbors': 29, 'p': 1}. Best is trial 0 with value: 0.3596358118361153.
[I 2025-08-27 20:19:47,985] Trial 1 finished with value: 0.4373016967857635 and parameters: {'n_neighbors': 12, 'p': 2}. Best is trial 1 with value: 0.4373016967857635.
[I 2025-08-27 20:20:07,724] Trial 2 finished with value: 0.4055731825079321 and parameters: {'n_neighbors': 23, 'p': 2}. Best is trial 1 with value: 0.4373016967857635.
[I 2025-08-27 20:20:33,208] Trial 3 finished with value: 0.4055731825079321 and parameters: {'n_neighbors': 23, 'p': 2}. Best is trial 1 with value: 0.4373016967857635.
[I 2025-08-27 20:21:08,996] Trial 4 finished with value: 0.4293005931852669 and parameters: {'n_neighbors': 14, 'p': 2}. Best is trial 1 with value: 0.4373016967857635.
[I 2025-08-27 20:21:34,779] Trial 5 finis

🏃 View run KNN_SMOTE_TFIDF_Trigrams_Corrected at: https://dagshub.com/satyajeetrai007/Youtube-Comment-Sentiment-Analysis.mlflow/#/experiments/6/runs/58e8960122964774a2c3d79f65026252
🧪 View experiment at: https://dagshub.com/satyajeetrai007/Youtube-Comment-Sentiment-Analysis.mlflow/#/experiments/6
